In [1]:
import os
import tensorflow
from tensorflow import keras
from matplotlib import pyplot as plt
import numpy as np
import random

In [2]:
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
import os
import seaborn as sns
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input

In [3]:
train_images=[]
train_labels=[]
val_images=[]
val_labels=[]
random_seed=42

In [4]:
print(os.listdir("/kaggle/input/animal-faces/afhq/train"))

['dog', 'wild', 'cat']


In [5]:
print('train data set')
train_cat_path='/kaggle/input/animal-faces/afhq/train/cat'
train_cat_img=[f for f in os.listdir(train_cat_path)]
print(f'고양이 사진: {len(train_cat_img)}개')

train_dog_path='/kaggle/input/animal-faces/afhq/train/dog'
train_dog_img=[f for f in os.listdir(train_dog_path)]
print(f'강아지 사진: {len(train_dog_img)}개')


train_wild_path='/kaggle/input/animal-faces/afhq/train/wild'
train_wild_img=[f for f in os.listdir(train_wild_path)]
print(f'다른동물 사진: {len(train_wild_img)}개')

train data set
고양이 사진: 5153개
강아지 사진: 4739개
다른동물 사진: 4738개


In [ ]:
min_samples = min(len(train_cat_img), len(train_dog_img), len(train_wild_img))

# 각 클래스에서 최소 샘플 수만큼 랜덤하게 선택하여 데이터셋 구성
selected_cat_img = random.sample(train_cat_img, min_samples)
selected_dog_img = random.sample(train_dog_img, min_samples)
selected_wild_img = random.sample(train_wild_img, min_samples)

train_set=[]
for img_file in selected_cat_img:
    img_path = os.path.join(train_cat_path, img_file)
    img = image.load_img(img_path, target_size=(224, 224))
    x = image.img_to_array(img)
    x = preprocess_input(x)
    train_set.append((x,0))  # 0은 고양이에 해당하는 라벨

for img_file in selected_dog_img:
    img_path = os.path.join(train_dog_path, img_file)
    img = image.load_img(img_path, target_size=(224, 224))
    x = image.img_to_array(img)
    x = preprocess_input(x)
    train_set.append((x,1))  # 1은 강아지에 해당하는 라벨

for img_file in selected_wild_img:
    img_path = os.path.join(train_wild_path, img_file)
    img = image.load_img(img_path, target_size=(224, 224))
    x = image.img_to_array(img)
    x = preprocess_input(x)
    train_set.append((x,2))  # 2는 다른 동물에 해당하는 라벨

random.shuffle(train_set)
# 리스트를 NumPy 배열로 변환
train_images = np.array([item[0] for item in train_set])
train_labels = np.array([item[1] for item in train_set])

In [ ]:
num_samples_to_visualize = 5

# 랜덤하게 몇 개의 인덱스 선택
random_indices = np.random.choice(len(train_images), num_samples_to_visualize, replace=False)

# 선택된 이미지와 라벨 시각화
for i in random_indices:
    img = train_images[i]
    label = train_labels[i]

    # 시각화
    plt.figure(figsize=(2, 2))
    plt.imshow(img)
    plt.title(f"Label: {label}")
    plt.axis('off')
    plt.show()

In [ ]:
print(f'train_image : {len(train_images)}개')
print(f'train_label : {len(train_labels)}개')

In [ ]:
print('validation data set')
valid_path='/kaggle/input/animal-faces/afhq/val'

valid_cat_path=os.path.join(valid_path, 'cat')
valid_cat_img=[f for f in os.listdir(valid_cat_path)]
print(f'고양이 사진: {len(valid_cat_img)}개')

valid_dog_path=os.path.join(valid_path, 'dog')
valid_dog_img=[f for f in os.listdir(valid_dog_path)]
print(f'강아지 사진: {len(valid_dog_img)}개')

valid_wild_path=os.path.join(valid_path, 'wild')
valid_wild_img=[f for f in os.listdir(valid_wild_path)]
print(f'다른동물 사진: {len(valid_wild_img)}개')

In [ ]:
valid_set=[]
for img_file in valid_cat_img:
    img_path = os.path.join(valid_cat_path, img_file)
    img = image.load_img(img_path, target_size=(224, 224))
    x = image.img_to_array(img)
    x = preprocess_input(x)
    valid_set.append((x,0))  # 0은 고양이에 해당하는 라벨

for img_file in valid_dog_img:
    img_path = os.path.join(valid_dog_path, img_file)
    img = image.load_img(img_path, target_size=(224, 224))
    x = image.img_to_array(img)
    x = preprocess_input(x)
    valid_set.append((x,1))  # 1은 강아지에 해당하는 라벨

for img_file in valid_wild_img:
    img_path = os.path.join(valid_wild_path, img_file)
    img = image.load_img(img_path, target_size=(224, 224))
    x = image.img_to_array(img)
    x = preprocess_input(x)
    valid_set.append((x,2))  # 2는 다른 동물에 해당하는 라벨

random.shuffle(train_set)
# 리스트를 NumPy 배열로 변환
valid_images = np.array([item[0] for item in valid_set])
valid_labels = np.array([item[1] for item in valid_set])

In [ ]:
print(len(valid_images))
print(len(valid_labels))

In [ ]:
random_indices = np.random.choice(len(valid_images), num_samples_to_visualize, replace=False)

# 선택된 이미지와 라벨 시각화
for i in random_indices:
    img = valid_images[i]
    label = valid_labels[i]

    # 시각화
    plt.figure(figsize=(2, 2))
    plt.imshow(img)
    plt.title(f"Label: {label}")
    plt.axis('off')
    plt.show()

In [ ]:
base_model=VGG16(weights='imagenet',include_top=True)

In [ ]:
base_model.summary()

In [ ]:
model = Sequential()
model.add(base_model)
model.add(Dense(3, activation='softmax'))
for layer in base_model.layers:
    layer.trainable = False

In [ ]:
model.summary()

In [ ]:
from keras.optimizers import Adam
model.compile(optimizer=Adam(lr=0.001), 
              loss='sparse_categorical_crossentropy', metrics=['accuracy', 'val_loss'])

In [ ]:
history = model.fit(train_images, train_labels, validation_data=(valid_images, valid_labels), epochs=10, batch_size=32)
test_loss, test_accuracy = model.evaluate(test_images, test_labels)
print(f'Test Loss: {test_loss}, Test Accuracy: {test_accuracy}')